In [40]:
import pandas as pd
import numpy as np
import datefinder
import datetime as dt
import matplotlib.pyplot as plt
import nltk
from nltk import sent_tokenize
from sklearn.ensemble import RandomForestClassifier
import joblib
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier
from pyproj import Proj, transform 
import re

In [42]:
nltk.download('punkt')

[nltk_data] Error loading punkt: HTTP Error 403: URLBlocked


False

In [5]:
path_dicc = "Diccionario.xlsx"
path_obs = "ObservacionesOTs.xlsx"
redes = "redes_alcantarillado.xlsx"

In [6]:
stop_words = pd.read_excel(path_dicc,sheet_name='Stopwords')
stop_words['Stopwords'] = stop_words['Stopwords'].astype(str)

In [7]:
stoplist = stop_words['Stopwords'].tolist()
vacias = []
for i in range(len(stoplist)):
  vacias.append(stoplist[i].upper())
print(vacias)

['&#X0D', '7M', 'A', 'ABAJO', 'ACCIONES', 'ACOSTA', 'ACTA', 'ACTIVIDAD', 'ACTIVIDADES', 'ADALBERTO', 'ADALVERTO', 'ADAN', 'ADICIONAL', 'ADOLFO', 'AGUDELO', 'AL', 'ALBERTO', 'ALCANTARILLADO', 'ALCANTARILLADOS', 'ALVAREZ', 'ALVARO', 'ANDEN', 'ANDÉN', 'ANDREA', 'ANGEL', 'ANS', 'ANTE', 'ANTIGUA', 'ANTONIO', 'APROX', 'APROXIMADAMENTE', 'ARANGO', 'ARBOLEDA', 'ARENAS', 'ARENILLA', 'ARGEMIRO', 'ARIAS', 'ARLEY', 'ARNULFO', 'ARRIBA', 'ARTURO', 'ASI', 'ASÍ', 'ATENCIÓN', 'AUGUSTO', 'AUXILIAR', 'BAJO', 'BASE', 'BASE&#X0D;', 'BEDOYA', 'BENITEZ', 'BERNARDO', 'BODEGAS', 'CABALLERO', 'CADAVID', 'CAMILA', 'CAMILO', 'CAMILO&#X0D', 'CANO', 'CARDONA', 'CARLOS', 'CARMONA', 'CASTRO', 'CELIS', 'CERCA', 'CESAR', 'CÉSAR', 'CHEQUEARLA', 'CLAVE', 'CON', 'CON&#X0D', 'CONCRETO', 'CONDICIONES', 'CONFIGURADO', 'CONTINUA', 'CONTRA', 'CONTRATO', 'CORREA', 'CORRESPONDIENTE', 'COSTADO', 'CUAL', 'CUÁL', 'CUALES', 'CUÁLES', 'CUENTA', 'CUESTA', 'DARIO', 'DAVID', 'DE', 'DEBIDO', 'DEFINA', 'DEFINE', 'DEFINIDO', 'DEFINIÓ', 'DE

In [8]:
sinonimos = pd.read_excel(path_dicc,sheet_name='Sinonimos')

In [10]:
def fechas(path):
  pattern = "\d{4}[/-]\d{2}[/-]\d{2}\s\d{1,2}:\d{1,2}:\d{1,2}"
  dates = re.findall(pattern, path)
  fechas = pd.DataFrame(dates,columns=['Fechas'])
  if len(fechas) > 1:
    fechas = fechas.sort_values('Fechas',ascending=False)
    ultima_fecha = fechas.iloc[0].values[0]
    penultima_fecha = fechas.iloc[1].values[0]
    indice_ultima = fechas.index[fechas['Fechas']==ultima_fecha]
    if indice_ultima == 0:
      new_line = path[:path.index(ultima_fecha)]
    else:
      new_line = path[path.index(penultima_fecha):path.index(ultima_fecha)]
  else:
    new_line = path
  return new_line

In [11]:
def quitar_numericos(path):
    patron='[0-9]+'
    return re.sub(patron, '', path)

In [12]:
def limpiar_caracteres(path):    
    patron = '[^A-Za-z0-9]+'
    return re.sub(patron, ' ', path)

In [13]:
def mayuscula(path):
  obs = path.upper()
  return obs

In [14]:
def token(path):
  words = nltk.word_tokenize(path)
  return words

In [15]:
def stopwords(path):
  text = [word for word in path if word not in vacias]
  return text

In [16]:
def quitar_tildes(path):
  a,b = 'áéíóúüñ','aeiouun'
  trans = str.maketrans(a,b)
  obs = path.translate(trans)
  return obs

In [17]:
def eliminar_ipid(path):
  return path.replace('ipid','')

In [18]:
def quitar_solicitud(path):
    return path.replace('solicitud','')

In [19]:
def palabras_vacias(path):
  sw = ['SI','RED','EL','LA','AECHEVEM','MMARINOR','JECHAVEC','YRUIZORT','RUBEN','XD','CL','LONDONO','MUNOZ','LEONOR','MARIA','TABORDA','SORLEY', 'MOSQUERA',
        'ES','CR','FUE','SU','NINGUNA','FOS','FOTOS','AALVARTA','ARRIETA','SANEAR','SROJASMA','KMORENM','LDELGAD','ARISTIZABAL','DUQUE','FOTOGRAFICO','REVISOR','GERARDO',
        'ANDRES','CHALA','WILBER','VEGA','GALEANO','DEBE']
  texto = [word for word in path if word not in sw]
  return texto

In [20]:
def reemplazar_palabras(path):
  for index in range(sinonimos.shape[0]):
    palabra = sinonimos.iloc[index, : ][0].lower()
    sinonimo = sinonimos.iloc[index, : ][1].lower()
    path = path.replace(palabra, sinonimo)
  return path

In [21]:
def tokenizacion(path):
  text = ' '.join(path)
  tokens = token(text)
  return tokens

In [22]:
def revertir_tokenizacion(path):
  text = ' '.join(path)
  return text

In [23]:
def extraer_ipid(path):
    posicion=path.find("IPID")
    if posicion != -1:
      num_ipid=path[posicion+5:posicion+12]
    else:
      num_ipid=-1
    return (num_ipid) 

In [24]:
redes_sigma = pd.read_excel(redes)
redes_sigma['IPID'] = redes_sigma['IPID'].astype(str)
list_ipid = redes_sigma['IPID'].tolist()

In [25]:
modelo_efectividad = joblib.load('modelo_entrenado_rforest.pkl')
modelo_causa = joblib.load('modelo_entrenado_naivebayes.pkl')

d:\Analitica\PNL_ordenes\pnl\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\Analitica\PNL_ordenes\pnl\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\Analitica\PNL_ordenes\pnl\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomizedSearchCV from version 1.0.2 when using version 1.1.2. This might lead to breaking code or inva

In [26]:
gesta = pd.read_excel(path_obs,sheet_name='Observaciones')
gesta_depurado = gesta.loc[:,['[$Dim Degenerada Ordenes].[ID Solicitud]','[$Dim Degenerada Ordenes].[Coordenada X]','[$Dim Degenerada Ordenes].[Coordenada Y]','[$Dim Degenerada Ordenes].[Cuadrilla]',
                           '[$Dim Degenerada Ordenes].[Observacion]','[$Efectividad].[Efectividad_1]','[$Efectividad].[Causa No Efectividad]','[$Elemento Infraestructura].[IPID]',
                           '[Ordenes].[Costo Total Definitivo]','[Ordenes].[Duracion Ejecución Horas Suma]']]
gesta_depurado.rename(columns={'[$Dim Degenerada Ordenes].[ID Solicitud]':'SOLICITUD','[$Dim Degenerada Ordenes].[Coordenada X]':'COORDENADA_X','[$Dim Degenerada Ordenes].[Coordenada Y]':'COORDENADA_Y',
                      '[$Dim Degenerada Ordenes].[Cuadrilla]':'CUADRILLA','[$Dim Degenerada Ordenes].[Observacion]':'OBSERVACION','[$Efectividad].[Efectividad_1]':'EFECTIVIDAD',
                      '[$Efectividad].[Causa No Efectividad]':'CAUSA_NO_EFECTIVIDAD','[$Elemento Infraestructura].[IPID]':'IPID',
                      '[Ordenes].[Costo Total Definitivo]':'COSTO_TOTAL','[Ordenes].[Duracion Ejecución Horas Suma]':'DURACION_HORAS'},inplace=True)
gesta_depurado['IPID'] = gesta_depurado['IPID'].astype(str)
gesta_depurado['DURACION_HORAS'] = gesta_depurado['DURACION_HORAS'].astype(float) 
gesta_depurado = gesta_depurado.merge(redes_sigma, left_on="IPID", right_on="IPID", how='left')
gesta_depurado['COOR_LAT'].fillna(0,inplace=True)
gesta_depurado['COOR_LON'].fillna(0,inplace=True)
gesta_depurado = gesta_depurado.tail(100)

In [28]:
def prediccion_efectividad(observacion):
  REPARA_TUBERIA = []
  TUBERIA_MAL = []
  PAVIMENTA_REPARA = []
  TUBERIA_PAVIMENTA = []
  CAMBIA_TUBERIA = []
  APIQUE_MARCA = []
  TUBERIA_REPARA = []
  HUECO_TUBERIA = []
  BUENO_FUNCIONA = []
  ATIENDE_OT = []
  GENERA_OT = []
  ACOMETIDA_MAL = []
  OT_ATIENDE = []
  CARPETA_ASFALTICA = []
  ATIENDE_DIRECCION = []
  DIRECCION_EVIDENCIA = []
  PAVIMENTA_DIRECCION = []
  DETERIORADA_CARPETA = []
  CAMARA_EMPUJE = []
  observacion_fecha = fechas(observacion)
  observacion_tildes = quitar_tildes(observacion_fecha)
  observacion_sinonimo = reemplazar_palabras(observacion_tildes)
  observacion_sinipid = eliminar_ipid(observacion_sinonimo)
  observacion_num = quitar_numericos(observacion_sinipid)
  observacion_depurada = limpiar_caracteres(observacion_num)
  observacion_mayuscula = mayuscula(observacion_depurada)
  observacion_token = token(observacion_mayuscula)
  observacion_limpia = stopwords(observacion_token)
  observacion_vacias = palabras_vacias(observacion_limpia)
  observacion_notoken = revertir_tokenizacion(observacion_vacias)
  VAL1 = [1 if 'REPARA TUBERIA' in observacion_notoken else 0]
  REPARA_TUBERIA.extend(VAL1)
  VAL2 = [1 if 'TUBERIA MAL' in observacion_notoken else 0]
  TUBERIA_MAL.extend(VAL2)
  VAL3 = [1 if 'PAVIMENTA REPARA' in observacion_notoken else 0]
  PAVIMENTA_REPARA.extend(VAL3)
  VAL4 = [1 if 'TUBERIA PAVIMENTA' in observacion_notoken else 0]
  TUBERIA_PAVIMENTA.extend(VAL4)
  VAL5 = [1 if 'CAMBIA TUBERIA' in observacion_notoken else 0]
  CAMBIA_TUBERIA.extend(VAL5)
  VAL6 = [1 if 'APIQUE MARCA' in observacion_notoken else 0]
  APIQUE_MARCA.extend(VAL6)
  VAL8 = [1 if 'TUBERIA REPARA' in observacion_notoken else 0]
  TUBERIA_REPARA.extend(VAL8)
  VAL9 = [1 if 'HUECO TUBERIA' in observacion_notoken else 0]
  HUECO_TUBERIA.extend(VAL9)
  VAL10 = [1 if 'BUENO_FUNCIONA' in observacion_notoken else 0]
  BUENO_FUNCIONA.extend(VAL10)
  VAL11 = [1 if 'ATIENDE OT' in observacion_notoken else 0]
  ATIENDE_OT.extend(VAL11)
  VAL12 = [1 if 'GENERA OT' in observacion_notoken else 0]
  GENERA_OT.extend(VAL12)
  VAL13 = [1 if 'ACOMETIDA MAL' in observacion_notoken else 0]
  ACOMETIDA_MAL.extend(VAL13)
  VAL14 = [1 if 'OT ATIENDE' in observacion_notoken else 0]
  OT_ATIENDE.extend(VAL14)
  VAL15 = [1 if 'CARPETA ASFALTICA' in observacion_notoken else 0]
  CARPETA_ASFALTICA.extend(VAL15)
  VAL16 = [1 if 'ATIENDE DIRECCION' in observacion_notoken else 0]
  ATIENDE_DIRECCION.extend(VAL16)
  VAL17 = [1 if 'DIRECCION EVIDENCIA' in observacion_notoken else 0]
  DIRECCION_EVIDENCIA.extend(VAL17)
  VAL18 = [1 if 'PAVIMENTA DIRECCION' in observacion_notoken else 0]
  PAVIMENTA_DIRECCION.extend(VAL18)
  VAL19 = [1 if 'DETERIORADA CARPETA' in observacion_notoken else 0]
  DETERIORADA_CARPETA.extend(VAL19)
  VAL20 = [1 if 'CAMARA EMPUJE' in observacion_notoken else 0]
  CAMARA_EMPUJE.extend(VAL20)

  dataframe_test=pd.DataFrame({"REPARA_TUBERIA":REPARA_TUBERIA,
                "TUBERIA_MAL":TUBERIA_MAL,
                "PAVIMENTA_REPARA":PAVIMENTA_REPARA,
                "TUBERIA_PAVIMENTA":TUBERIA_PAVIMENTA,
                "CAMBIA_TUBERIA":CAMBIA_TUBERIA,
                "HUECO_TUBERIA":HUECO_TUBERIA,
                "APIQUE_MARCA":APIQUE_MARCA,
                "TUBERIA_REPARA":TUBERIA_REPARA,
                "HUECO_TUBERIA":HUECO_TUBERIA,
                "BUENO_FUNCIONA":BUENO_FUNCIONA,
                "ATIENDE_OT":ATIENDE_OT,
                "GENERA_OT":GENERA_OT,
                "ACOMETIDA_MAL":ACOMETIDA_MAL,
                "OT_ATIENDE":OT_ATIENDE,
                "CARPETA_ASFALTICA":CARPETA_ASFALTICA,
                "ATIENDE_DIRECCION":ATIENDE_DIRECCION,
                "DIRECCION_EVIDENCIA":DIRECCION_EVIDENCIA,
                "PAVIMENTA_DIRECCION":PAVIMENTA_DIRECCION,
                "DETERIORADA_CARPETA":DETERIORADA_CARPETA,
                "CAMARA_EMPUJE":CAMARA_EMPUJE,})
  efectividad_pred = modelo_efectividad.predict(dataframe_test)[0]
  if efectividad_pred == 0:
    valor = "NO"
  else:
    valor = "SI"
  return(valor)

In [29]:
def comparacion_ipid(observacion, ipid_desc, efec):
  ipid_obs = extraer_ipid(observacion)
  if ipid_obs != -1:
    if efec != 'NO':
      if ipid_obs != ipid_desc:
        resultado = "Diferente"
      else:
        resultado = "Mismo ipid"
    else:
      resultado = "No aplica"
  else:
    resultado = "No aplica"
  return(resultado)

In [30]:
def comparacion_listaipid(efectividad,ipid):
  if efectividad == "SI":
    if ipid in list_ipid:
      resultado = "está"
    else:
      resultado = "no está"
  else:
    resultado = "no aplica"
  return(resultado)

In [34]:
def calculo_distancia(coordenada_x, coordenada_y,coor_lat,coor_long):
  inProj = Proj(init='epsg:4236') 
  outProj = Proj(init='epsg:3116')
  if coor_lat > 0:
    x1,y1= transform(inProj,outProj,coordenada_x,coordenada_y)
    x2,y2= transform(inProj,outProj,coor_long,coor_lat)
    xdis = abs(x1 - x2)
    ydis = abs(y1 - y2)
    distancia = np.sqrt((xdis**2)+(ydis**2))
  else:
    distancia = 0
  return(distancia)

In [35]:
def diferencia_costo(efectividad, costo):
  if efectividad == "SI":
    if costo < 100000:
      inconsistencia = "SI"
    else:
      inconsistencia = "NO"
  else:
      inconsistencia = "NO"
  return(inconsistencia)

In [36]:
def control_tiempo(efectividad, tiempo):
    if efectividad == "SI":
      if tiempo < 0.5:
        inconsistencia = "SI"
      else:
        inconsistencia = "NO"
    else:
        inconsistencia = "NO"
    return(inconsistencia)

In [37]:
def causa_noefec(observacion_ultima):
  resultado = modelo_causa.classify(observacion_ultima)
  return(resultado)

In [38]:
def main(path):
  indicador_lista = []
  efectividad_prediccion = []
  comparacion_ipid_obs = []
  causa_prediccion = []
  ipid_modelored = []
  lista_distancias = []
  resultado_costo = []
  resultado_tiempo = []
  for index in range(path.shape[0]):
    indicador = 0
    coordenada_y = path.iloc[index, : ][2]
    coordenada_x = path.iloc[index, : ][1]
    observacion = path.iloc[index, : ][4]
    observacion_ultima = fechas(observacion)
    efectividad = path.iloc[index, : ][5]
    causa_noefectividad = path.iloc[index, : ][6]
    ipid_descargado = path.iloc[index, : ][7]
    costo = path.iloc[index, : ][8]
    duracion = path.iloc[index, : ][9]
    coor_lat = path.iloc[index, : ][10]
    coor_long = path.iloc[index, : ][11]
    ##Prediccion efectividad
    efectividad_pred = prediccion_efectividad(observacion)
    efectividad_prediccion.append(efectividad_pred)
    if efectividad_pred != efectividad:
      indicador += 1
    else:
      pass
    ##Prediccion causa no efectividad
    if efectividad == "NO":
      causa_pred = modelo_causa.classify(observacion_ultima)
      causa_prediccion.append(causa_pred)
      if causa_pred != causa_noefectividad:
        indicador += 1
      else:
        pass
    else:
      causa_prediccion.append("NA")
    ##Comparar ipid
    resultado_compipid = comparacion_ipid(observacion,ipid_descargado,efectividad)
    comparacion_ipid_obs.append(resultado_compipid)
    if resultado_compipid == "Diferente":
      indicador += 1
    else:
      pass
    ##Comparar ipid modelo de red
    resultado_modelored = comparacion_listaipid(efectividad, ipid_descargado)
    ipid_modelored.append(resultado_modelored)
    if resultado_modelored == "no está":
      indicador += 2
    else:
      pass
    #Diferencia costo
    resultado_dif_costo = diferencia_costo(efectividad, costo)
    resultado_costo.append(resultado_dif_costo)
    if resultado_dif_costo == "SI":
      indicador += 1
    else:
      pass
    #Control tiempo
    diferencia_tiempo = control_tiempo(efectividad, duracion)
    resultado_tiempo.append(diferencia_tiempo)
    if diferencia_tiempo == "SI":
      indicador += 1
    else:
      pass
    #Control distancias
    resultado_distancia = calculo_distancia(coordenada_x, coordenada_y,coor_lat,coor_long)
    lista_distancias.append(resultado_distancia)
    if resultado_distancia >= 100:
      indicador += 1
    else:
      pass
    indicador_lista.append(indicador)
  path['Calidad_ot'] = ["Buena" if indicador>0 else "Mala" for indicador in path['indicador']]
  gesta_depurado['Value'] = 1
  path['efectividad_predecida'] = efectividad_prediccion
  path['causa_noefectividad_predecida'] = causa_prediccion
  path['comparacion_ipid_obs'] = comparacion_ipid_obs
  path['ipid_modelored'] = ipid_modelored
  path['distancias_ipid'] = lista_distancias
  path['inconsistencia_costo'] = resultado_costo
  path['inconsistencia_tiempo'] = resultado_tiempo
  path['indicador'] = indicador_lista
  return(path.head(10))

In [41]:
main(gesta_depurado)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\squintra/nltk_data'
    - 'd:\\Analitica\\PNL_ordenes\\pnl\\nltk_data'
    - 'd:\\Analitica\\PNL_ordenes\\pnl\\share\\nltk_data'
    - 'd:\\Analitica\\PNL_ordenes\\pnl\\lib\\nltk_data'
    - 'C:\\Users\\squintra\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************
